## Land Cover Classification: Sentinel-2 with Cropland Data Layer Annotations

This tutorial will walk you through the process of performing land cover classification with Sentinel-2 satellite imagery, annotated with CDL (Cropland Data Layer), using TorchGeo library. We will start with downloading Sentinel-2 data, setting it up with CDL for pixel-wise supervised classification, training a segmentation model, and finally running inference to make sense of Sentinel-2 imagery. 

Whether you are a remote sensing enthusiast or just curious about deep learning for geospatial data, this guide has something cool for you!

In [28]:
import os
from urllib.parse import urlparse

import matplotlib.pyplot as plt
import planetary_computer
import pystac
import torch
from torch.utils.data import DataLoader
from torchgeo.datasets import RasterDataset
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)

### 1- Downloading Sentinel-2 Imagery

Fetch Sentinel-2 imagery using Microsoft Planetary Computer and ensure you have the data you need.

In [ ]:
root = "/data/sentinel"
item_urls = [
    'https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2B_MSIL2A_20241101T170349_R069_T15TWG_20241101T204038',
]

for item_url in item_urls:
    item = pystac.Item.from_file(item_url)
    signed_item = planetary_computer.sign(item)
    for band in ['B02', 'B03', 'B04', 'B08']:
        asset_href = signed_item.assets[band].href
        filename = urlparse(asset_href).path.split('/')[-1]
        download_url(asset_href, root, filename)

### 2-  Prepare Dataloader

Customize TorchGeo to align `Sentinel2` and `CDL` datasets, forming an `IntersectionDataset` for pixel-wise classification task

In [29]:
from torchgeo.datasets import Sentinel2

class Sentinel2Custom(Sentinel2):
    filename_glob = 'T*_B02_10m.tif'
    filename_regex = r'.*'
    date_format = '%Y%m%dT%H%M%S'
    is_image = True
    separate_files = True
    all_bands = ('B02', 'B03', 'B04', 'B08')
    rgb_bands = ('B04', 'B03', 'B02')

import torchgeo.datasets
torchgeo.datasets.Sentinel2 = Sentinel2Custom

from torchgeo.datamodules import Sentinel2CDLDataModule

datamodule = Sentinel2CDLDataModule(
    cdl_crs="epsg:4326",
    sentinel2_crs="epsg:4326",
    cdl_paths="/data/datatorchgeo",
    sentinel2_paths="/data/sentinel",
    batch_size=1,
    patch_size=64,
    length=8
)

datamodule.setup("fit")
train_dataset = datamodule.train_dataset
datamodule.setup("validate")
val_dataset = datamodule.val_dataset
datamodule.setup("test")
test_dataset = datamodule.test_dataset

datamodule.val_sampler.length = 3
datamodule.test_sampler.length = 3

Converting CDL res from 0.000325900410757084 to 10
Converting CDL res from 0.000325900410757084 to 10
Converting CDL res from 0.000325900410757084 to 10


### 3- Training Semantic Segmentation Model

Train a UNet model with Sentinel-2 images paired with CDL labels to classify land cover, powered by PyTorch Lightning

In [30]:
from torchgeo.trainers import SemanticSegmentationTask
from lightning.pytorch import Trainer
import torch

task = SemanticSegmentationTask(
    model='unet',
    backbone='resnet50',
    weights=None,  # No pre-trained weights
    in_channels=3,  # CDL dataset may have RGB inputs
    num_classes=134,
    num_filters=3,
    loss='ce',  # CrossEntropyLoss
    class_weights=None,
    ignore_index=None,
    lr=0.001,
    patience=10,
    freeze_backbone=False,
    freeze_decoder=False)

accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'

trainer = Trainer(
    accelerator=accelerator,
    default_root_dir='./',
    fast_dev_run=True,
    log_every_n_steps=1,
    min_epochs=1,
    max_epochs=2,
)

trainer.fit(model=task, datamodule=datamodule)

INFO: Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU av

Converting CDL res from 0.000325900410757084 to 10


MisconfigurationException: Sentinel2CDLDataModule.val_sampler has length 0.

### 4- Model Evaluation on Test Set

Evaluate the segmentation model over the test set

### 5- Inference over full Imagery

Perform inference by computing predictions over the complete imagery 

### Debug

self.length = 0 is not populating in gridgeosample

self.length = 0
self.hits = []
areas = []
for hit in self.index.intersection(tuple(self.roi), objects=True):
    bounds = BoundingBox(*hit.bounds)
    if (
        bounds.maxx - bounds.minx >= self.size[1]
        and bounds.maxy - bounds.miny >= self.size[0]
    ):
        if bounds.area > 0:
            rows, cols = tile_to_chips(bounds, self.size)
            self.length += rows * cols
        else:
            self.length += 1
        self.hits.append(hit)
        areas.append(bounds.area)
if length is not None:
    self.length = length

In [24]:
datamodule.val_dataset

In [3]:
len(train_dataset), len(val_dataset), len(test_dataset)

(52, 6, 6)

In [4]:
datamodule.sentinel2.files

['/data/sentinel/T15TWG_20241101T170349_B02_10m.tif']

In [5]:
datamodule.cdl.files

['/data/datatorchgeo/2023_30m_cdls.tif']

In [6]:
datamodule.train_dataset.bounds

BoundingBox(minx=-93.00024322601918, maxx=-91.66508508222715, miny=41.456209388641454, maxy=42.45269119869629, mint=1672531200.0, maxt=1704067199.999999)

In [7]:
datamodule.sentinel2.bounds

BoundingBox(minx=-93.00024322601918, maxx=-91.66508508222715, miny=41.456209388641454, maxy=42.45269119869629, mint=0.0, maxt=9.223372036854776e+18)

In [8]:
datamodule.cdl.bounds

BoundingBox(minx=-127.88721217969017, maxx=-65.34561975376272, miny=22.94022503977174, maxy=51.60512156832182, mint=1672531200.0, maxt=1704067199.999999)

In [9]:
datamodule.train_dataset.datasets

### intersection dataset: 
        self.dataset = self.sentinel2 & self.cdl


In [10]:
datamodule.dataset

In [11]:
datamodule.train_dataset

In [12]:
datamodule.train_dataset, len(datamodule.train_dataset), len(datamodule.val_dataset), len(datamodule.test_dataset)

(<torchgeo.datasets.geo.IntersectionDataset at 0x7fb9a2912ed0>, 52, 6, 6)

In [13]:
datamodule.train_batch_sampler.size

(640, 640)

In [14]:
from torchgeo.datasets.utils import BoundingBox

for i in datamodule.cdl.index.intersection(datamodule.cdl.index.bounds, objects=True):
    box1 = BoundingBox(*i.bounds)
    print(box1.area)

1792.7482756199618


In [15]:
for i in datamodule.sentinel2.index.intersection(datamodule.sentinel2.index.bounds, objects=True):
    box1 = BoundingBox(*i.bounds)
    print(box1.area)

1.3304608038353392


In [16]:
i = 0

for hit1 in datamodule.cdl.index.intersection(datamodule.cdl.index.bounds, objects=True):
    print("hit1", hit1)
    for hit2 in datamodule.sentinel2.index.intersection(hit1.bounds, objects=True):
        box1 = BoundingBox(*hit1.bounds)
        box2 = BoundingBox(*hit2.bounds)
        print("box1.area", box1.area)
        print("box2.area", box2.area)
        box3 = box1 & box2
        # Skip 0 area overlap (unless 0 area dataset)
        if box3.area > 0 or box1.area == 0 or box2.area == 0:
            # self.index.insert(i, tuple(box3))
            i += 1

if i == 0:
    raise RuntimeError('Datasets have no spatiotemporal intersection')

hit1 <rtree.index.Item object at 0x7fb9a291b7e0>
box1.area 1792.7482756199618
box2.area 1.3304608038353392


### Debug Sentinel2CDLDataModule.train_batch_sampler has len 0

In [17]:
datamodule.train_batch_sampler.size

(640, 640)

In [18]:
datamodule.train_dataset, datamodule.patch_size, datamodule.batch_size, datamodule.length

(<torchgeo.datasets.geo.IntersectionDataset at 0x7fb9a2912ed0>, 64, 1, 8)

In [19]:
datamodule.length

8

In [20]:
len(datamodule.train_batch_sampler), len(datamodule.val_sampler), len(datamodule.test_sampler)

(8, 3, 3)